## Roark's Formulas - Bending calculations chapter

Ref: Chapter 9, tab 8.1, formula a

Roark's Formulas for Stress and Strain

CHAPTER 8 Beams; Flexure of Straight Bars

TABLE 8.1 Shear, Moment, Slope, and Deflection Formulas for Elastic Straight Beams

1. Concentrated intermediate load

1a. Left end free, right end fixed (cantilever)

Rev: 0

Date: February. 18th, 2022

Made by: Minh LE

In [1]:
# Python librairies imports
# import plotly as py
# import plotly.graph_objs as go
# import matplotlib.pyplot as plt
# import plotly.express as px
from ipywidgets import FloatSlider, HBox, VBox
import bqplot.pyplot as plt
from ipywidgets import *
from bqplot import Toolbar

import numpy as np

from IPython.display import Markdown as md
from IPython.display import Image, display


### I) Input
<em>Notation</em>: $W$ = load (force); $w$ =unit load (force per unit length); $M_0$ = applied couple (force-length); $\theta_0$ externally created concentrated angular displacement (radians); $\Delta_0$ = externally created concentrated lateral displacement; $T_1$ and $T_2$ = temperatures on the top and bottom surfaces, respectively (degrees). $R_A$ and $R_B$ are the vertical end reactions at the left and right, respectively, and are positive upward. $M_A$ and $M_B$ are the reaction end moments at the left and right, respectively. All moments are positive when producing compression on the upper portion of the beam cross section. The transverse shear force $V$ is positive when acting upward on the left end of a portion of the beam. All applied loads, couples, and displacements are positive as shown.

All deflections are positive upward, and all slopes are positive when up and to the right. $E$ is the modulus of elasticity of the beam material, and $I$ is the area moment of inertia about the centroidal axis of the beam cross section. $\gamma$ is the temperature coefficient of expansion (unit strain per degree).

![Left end free, right end fixed (cantilever)](images_lib/case1_page203.png)

Users need to specify principal information, parameters and dimensions.
![Left end free, right end fixed (cantilever)](images_lib/case1a_page203.png)

In [3]:
W = 150. #(lb)
a = 0. #(in)
l = 40. #(in)
E = 1300000. # (lb/in2)
I = (2. * 4.**3.)/12. #(in4)

In [4]:
class compute_tab8p1a:
    def __init__(self, a=a, W=W, l=l, E=E, I=I):
        self.W = W
        self.a = a
        self.l = l
        self.E = E
        self.I = I
        
    def compute_tab8p1a_fixValues(self):
        # Boundary Values
        self.R_A = 0.
        self.M_A = 0.
        self.theta_A = self.W*(self.l-self.a)**2. / (2.*self.E*self.I)
        self.y_A = -(self.W/(6.*self.E*self.I)) * (2.*self.l**3.-3*self.l**2.*self.a+self.a**3.)
        #
        self.R_B = self.W
        self.M_B = -self.W*(self.l-self.a)
        self.theta_B = 0.
        self.y_B = 0.
        # Selected Maximum Values of Moments and Deformations
        self.M_max = self.M_B   #Max moment, found at B
        self.theta_max = self.theta_A   #Max slope, found at A
        self.y_max = self.y_A   #Max deflection, found at A
        self.M_max_fa = -self.W*self.l   #Max moment, found at B, when a=0, (fa signifie f(a))
        self.theta_max_fa = self.W*self.l**2./(2.*self.E*self.I)   #Max slope, found at A, when a=0, (fa signifie f(a))
        self.y_max_fa = -(self.W*self.l**3.)/(3.*self.E*self.I)   #Max deflection, found at A, when a=0, (fa signifie f(a))
        return self.y_A, self.theta_A, self.M_A, self.R_A, self.y_max_fa, self.theta_max_fa, self.M_max_fa, self.M_B

    def list_tab8p1a_values(self):
        self.compute_tab8p1a_fixValues()
        # Selected Maximum Values of Moments and Deformations
        print('Max moment, found at B: \n', self.M_B, '\n')
        print('Max slope, found at A: \n', self.theta_A, '\n')
        print('Max deflection, found at A: \n', self.y_A, '\n')

    def compute_tab8p1a_deflect(self, x):
        self.compute_tab8p1a_fixValues()
        self.y_x = self.y_A + self.theta_A*x + self.M_A*x**2./(2.*self.E*self.I) + self.R_A*x**3./(6.*self.E*self.I) - \
                    self.W*(x-self.a)**3.*np.heaviside((x-self.a),0.)/(6.*E*I) #Deflection
        return self.y_x

    def draw_tab8p1a_deflect(self):
        x=np.linspace(0,l,200) #Array
        y0=np.linspace(0,0,200) #Zero array
        y = self.compute_tab8p1a_deflect(x)
        xy_plot(
            ('Deflection', x, y, {"mode": 'lines'}),  
            ('y = 0', x, y0, {"mode": 'lines', "color": "black"}),
            x_label='Length in ft', y_label='Deflection in lb.ft', 
            title=f'Case 1a. Left end free, right end fixed (cantilever), length {l} m\
            <br>Concentrated force {self.W} N applied at {self.a} m from free end', 
            annotation='',
        ).show()

    def compute_tab8p1a_shear(x, a=a, W=W):
        y_A, theta_A, M_A, R_A, y_max_fa, theta_max_fa, M_max_fa, M_B = compute_tab8p1a_fixValues(a=a, W=W)
        V_x = R_A-W*np.heaviside((x-a),0.) #Transverse shearn
        return V_x

    def compute_tab8p1a_moment(self, x):
        self.compute_tab8p1a_fixValues()
        self.M_x = self.M_A+self.R_A*x-self.W*(x-self.a)*np.heaviside((x-self.a),0.) #Bending Moment
        return self.M_x

    def draw_tab8p1a_moment(self):
        x=np.linspace(0,l,200) #Array
        y0=np.linspace(0,0,200) #Zero array
        y = self.compute_tab8p1a_moment(x)
        xy_plot(
            ('Bending Moment', x, y, {"mode": 'lines'}),  
            ('y = 0', x, y0, {"mode": 'lines', "color": "black"}),
            x_label='Length in ft', y_label='Bending Moment in lb.ft', 
            title=f'Case 1a. Left end free, right end fixed (cantilever), length {l} m\
            <br>Concentrated force {W} N applied at {a} m from free end', 
            annotation='',
        ).show()

    def compute_tab8p1a_slope(x, a=a, W=W):
        y_A, theta_A, M_A, R_A, y_max_fa, theta_max_fa, M_max_fa, M_B = compute_tab8p1a_fixValues(a=a, W=W)
        theta_x = theta_A + M_A*x/(E*I) + R_A*x**2./(2.*E*I) - W*(x-a)**2.*np.heaviside((x-a),0.)/(2.*E*I) #Slope
        return theta_x

### II) Results: Values and graphic plots
Selected results and plots

#### Selected Maximum Values of Moments and Deformations

In [5]:
compute_tab8p1a().list_tab8p1a_values()

Max moment, found at B: 
 -6000.0 

Max slope, found at A: 
 0.008653846153846154 

Max deflection, found at A: 
 -0.23076923076923075 



#### Interactive Deflection plots with slider W and a

In [8]:
x = np.linspace(0., l, 200)
y = compute_tab8p1a().compute_tab8p1a_deflect(x)

# plot the deflection
title_tmpl = 'Bending (W = {} and a = {})'
pdf_fig = plt.figure(title=title_tmpl.format(150., 0.))
pdf_fig.layout.width = '600px'
pdf_fig.layout.height = '400px'
pdf_line = plt.plot(x, y, 'm', stroke_width=3)

# use two sliders to represent W and a
W_slider = FloatSlider(description='$W$', value=150, min=0, max=300, step=10.)
a_slider = FloatSlider(description='$a$', value=0, min=0, max=l, step=1.)
slider_layout = HBox([W_slider, a_slider])

def update_deflect_aW(change):
    new_a = a_slider.value
    new_W = W_slider.value
    pdf_line.y = compute_tab8p1a(a=new_a, W=new_W).compute_tab8p1a_deflect(x)
    pdf_fig.title = title_tmpl.format(new_W, new_a)

# register the above callback with the 'value' trait of the sliders
for slider in [W_slider, a_slider]:
    slider.observe(update_deflect_aW, 'value')

# VBox([pdf_fig, slider_layout])

In [9]:
txt_layout = Layout(width='200px')
E_box = FloatText(description='$E$', 
                    style={'description_width':'initial'}, 
                    layout=txt_layout, value=E)
I_box = FloatText(description='$I$', 
                    layout=txt_layout, value=I)
l_box = FloatText(description='$l$', 
                    layout=txt_layout, value=l)

go_btn = Button(description='GO', button_style='success', layout=Layout(width='50px'))

def update_deflect_lEI():
    #pdf_line.y = compute_tab8p1a(l=l_box.value, E=E_box.value, I=I_box.value).compute_tab8p1a_deflect(x)
    x = np.linspace(0., l_box.value, 200)
    a_slider.max = l_box.value
    slider_layout = HBox([W_slider, a_slider])
    pdf_line.x = x
    pdf_line.y = compute_tab8p1a(l=l_box.value, E=E_box.value, I=I_box.value).compute_tab8p1a_deflect(x)
    def update_deflect_aW(change):
        new_a = a_slider.value
        new_W = W_slider.value
        pdf_line.y = compute_tab8p1a(a=new_a, W=new_W, l=l_box.value, E=E_box.value, I=I_box.value).compute_tab8p1a_deflect(x)
        pdf_fig.title = title_tmpl.format(new_W, new_a)

    # register the above callback with the 'value' trait of the sliders
    for slider in [W_slider, a_slider]:
        slider.observe(update_deflect_aW, 'value')

go_btn.on_click(lambda btn: update_deflect_lEI())

In [10]:
# left_box = VBox([slider_layout])
left_box = VBox([pdf_fig, slider_layout])
right_box = VBox([E_box, I_box, l_box, go_btn])
HBox([left_box, right_box])